<a href="https://colab.research.google.com/github/kkhendry/Benchmarking-Quantum-Groundstate-Solvers/blob/master/main/BenchmarkingTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To work using VSCode Editor, run cell 1 and 2 below.
Input your own ngrok authentication token in cell 1 and access the VSCode ngrok tunnel by clicking the link in cell 2.

In [ ]:
#@title Cell 1
#%%capture uncomment in google colab to hide output
!pip install colabcode
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip
!./ngrok authtoken 20U3834NIEtDeUcn98AGuj8rL7B_3trunaPVasZKZijiuZUCH #add your own authtoken here. Log in/sign up for an ngrok account at https://ngrok.com/.
from colabcode import ColabCode


In [ ]:
#@title Cell 2
ColabCode(port=10000)

To work from google colab, run cell 3. This option is great for running to notebook, but limited for editing helper and utility files and functions.

In [ ]:
#@title Cell 3
#%%capture uncomment in google colab to hide output
!git clone https://github.com/kkhendry/QOSF-2021-Cohort-4-Hydra.git
import sys
sys.path.insert(0, '/content/QOSF-2021-Cohort-4-Hydra/')
%cd QOSF-2021-Cohort-4-Hydra

Run cell 4 and 5 below to set up all the requirements to run the rest of the notebook. When dwave setup prompts you for an authentication token in cell 5, please use your own. You can sign up for a dwave leap account with limited usage at https://cloud.dwavesys.com/leap/login/?next=/leap/.

In [ ]:
#@title Cell 4
#%%capture uncomment in google colab to hide output. if using vscode, simply copy and paste the code below in the terminal. For git config add your own email address and user name associated with your github account.
!git config --global user.email karan.khendry@gmail.com
!git config --global user.name hydraken
!pip install -r setup

In [ ]:
#@title Cell 5
#%%capture uncomment in google colab to hide output. if using vscode, simply copy and past the code below in the terminal.
#API Token: DEV-4f5f3a7d07f2dc98dccd35b17cdbaf20b5b7e8cf #use your own dwave Leap API Token
!dwave setup

Install all required modules in cell 6 below.

In [ ]:
#@title Cell 6
#General
import numpy as np
import scipy
import scipy.linalg
import math
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

#Molecule Drivers
import pyscf
from pyscf import gto, scf, mcscf

##QISKIT
#Molecule, Fermioninc Operator, Problem Definition
from qiskit_nature.drivers import Molecule
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

#Qubit Operator
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization import QubitConverter

#Circuits, Simulators, Algorithms, Solvers and VQE
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2, TwoLocal, NLocal, PauliTwoDesign
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import VQEUCCFactory, GroundStateEigensolver

##OPENFERMION
from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy
import scipy
import scipy.linalg

##UTILITIES
from helper_functions import *
from XBK_method import *
from QCC_method import *

Set up the molecule and electronic structure rpoblem to work with in cell 7 below. Some molecule have been set up below, uncomment the relevant sections to run experiements with those or create an equivalent code block with a molecule of your choice.

In [ ]:
#@title Cell 6

#H2
# geometry = [['H', [  0.0,    0.0,    0.0]],
#             ['H', [  0.0,    0.0,   0.739]]]
# molecule = Molecule(geometry=geometry, charge=0, multiplicity=1)
# molecule_name = 'H2'
# basis = 'sto6g'

# H3
# geometry = [['H', [0.0, 0.0, 0.0]], ['H', [1.1, 0.0, 0.0]], ['H', [0.55, 0.9526279441628825, 0]]]
# molecule = Molecule(geometry=geometry, charge=1, multiplicity=1)
# molecule_name = 'H3'
# basis = 'sto3g'

# #LiH
geometry = [['Li', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 1.5474]]]
molecule = Molecule(geometry=geometry, charge=0, multiplicity=1)
molecule_name = 'LiH'
basis = 'sto3g'

# # H20
# geometry = [['O', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 1.0]], ['H', [0.0, 1.0, 0.0]]]
# molecule = Molecule(geometry=geometry, charge=0, multiplicity=1)
# molecule_name = 'H20'
# basis = 'sto3g'

# #HCONH2
# geometry = [["O",  [1.1280, 0.2091, 0.0000]],
#             ["N", [-1.1878, 0.1791, 0.0000]],
#             ["C", [0.0598, -0.3882, 0.0000]],
#             ["H", [-1.3085, 1.1864, 0.0001]],
#             ["H", [-2.0305, -0.3861, -0.0001]],
#             ["H", [-0.0014, -1.4883, -0.0001]],
#             ['C', [-0.1805, 1.3955, 0.000]]]
# molecule = Molecule(geometry=geometry, charge=0, multiplicity=1)
# molecule_name = 'HCONH2'
# basis = 'sto3g'

class Problem: 
    def __init__(self, molecule, molecule_name, basis):
        self.molecule = molecule
        self.molecule_name = molecule_name
        self.basis = basis

    def problemOperators(self, exact_solution = True):
        driver = ElectronicStructureMoleculeDriver(molecule=self.molecule, basis=self.basis, driver_type=ElectronicStructureDriverType.PYSCF)

        self.es_problem = ElectronicStructureProblem(driver)
        self.second_q_op = self.es_problem.second_q_ops()
        self.q_fermionic_op = self.second_q_op[0]
        # register_length = q_fermionic_op.register_length
        self.of_fermionic_op = qiskit2of_fermionicOp(self.q_fermionic_op)

        self.qubit_converter = QubitConverter(JordanWignerMapper())
        self.q_qubit_op = self.qubit_converter.convert(self.q_fermionic_op)
        self.of_qubit_op = jordan_wigner(self.of_fermionic_op)
        # m = count_qubits(qubit_H)
        # print(qubit_H)

        if exact_solution:
            numpy_solver = NumPyMinimumEigensolver()
            numpy_ground_state_solver = GroundStateEigensolver(self.qubit_converter, numpy_solver)
            numpy_results = numpy_ground_state_solver.solve(self.es_problem)
            self.exact_energy = numpy_results.computed_energies[0]
            print(f"Exact electronic energy: {self.exact_energy:.6f} Hartree\n")
            # print(numpy_results)

def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

def minEnergyResults(result_dict, method):
    fig, ax1 = plt.subplots(1, 1)
    fig.set_figwidth(12)
    fig.set_figheight(9)
    color = 'tab:blue'
    ax1.set_xlabel('Iterations')
    ax1.set_ylabel('Energy')
    ax1.plot(range(len(result_dict['min energies'])), result_dict['min energies'])
    ax1.axhline(result_dict['exact energy'], linestyle='--')

    # ax2 = ax1.twinx() 
    # color = 'tab:orange'
    # ax2.set_ylabel('Time', color=color)
    # ax2.bar(range(len(min_energies)), iter_times, color=color, alpha = 0.2, width = 0.2)

    if method == 'VQE':
        fig_title = f"\
        {result_dict['setup']['mapping']}-\
        {result_dict['setup']['ansatz']}-\
        {result_dict['setup']['optimizer']}"
        plt.title(method+" Convergence Plot\n"+fig_title)
    elif method == 'QCC':
        fig_title = f"\
        {result_dict['setup']['mapping']}-\
        {result_dict['setup']['angle_folds']}-\
        {result_dict['setup']['amplitude_folds']}"
        plt.title(method+" Convergence Plot\n"+fig_title)
    elif method == 'XBK':
        fig_title = f"\
        {result_dict['setup']['mapping']}-\
        {result_dict['setup']['r']}"
        plt.title(method+" Convergence Plot\n"+fig_title)

    newline = '\n'
    fig.text(0.65, 0.75, f"\
        Minimum Energy: {result_dict['result energy (Ha)']:.3f}{newline}\
        Runtime: {result_dict['runtime']:.3f}{newline}\
        Number of Iterations: {result_dict['# of evaluations']}")
    
    with open('experiments/results.json', 'r') as file:
        results = json.load(file)

    results.update(result_dict)

    with open('experiments/results.json','w') as file:
        json.dump(results, file, sort_keys=True, indent=4)

problem = Problem(molecule, molecule_name, basis)
mol_dict = {'molecule name': problem.molecule_name, 'basis': problem.basis}
problem.problemOperators()


Exact electronic energy: -8.908697 Hartree



Set up the IBM Quantum Device backend in cell 7 below. DO not run cell 7 if you would like to use a simulator. Please use your own IBM Quantum authentication API token. You can login/sign up for an IBM Quantum account at https://quantum-computing.ibm.com/

Run cell 8 below to set up the quantum annealer simulator or device. Uncomment as applicable.

In [ ]:
#@title Cell 8
# from neal import SimulatedAnnealingSampler
# sampler = SimulatedAnnealingSampler() #uses simulated annealing, see D-Wave's ocean sdk for more options

from dwave.system import LeapHybridSampler
sampler = LeapHybridSampler()

Run cell 9 to get the classical solution for the exact electronic energy, this may not be feasible for large molecules. (Isn't that the point?)

In [ ]:
method = "VQE"
init_state = HartreeFock(problem.es_problem.num_spin_orbitals, problem.es_problem.num_particles, problem.qubit_converter)

## Backend
from qiskit import IBMQ
IBMQ.save_account('f10a5be117c16a6472c748a25874a94883f6021cf9bfe6f406e015522235a968d8eb0a5812f47160385c72355172371278d3c36e75e23fc7f4d73f1523fcbdf8')
provider = IBMQ.load_account()
# provider = IBMQ.get_provider(project='qiskit-runtime')  # Change this to your provider.
backend = provider.get_backend('ibmq_qasm_simulator')
# backend = QasmSimulator()

## Ansatz
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

ansatz_type = "UCCSD"
if ansatz_type == "TwoLocal":
    rotation_blocks = ['ry', 'rz']
    entanglement_blocks = 'cx'
    entanglement = 'full'
    repetitions = 3
    skip_final_rotation_layer = True
    ansatz = TwoLocal(problem.q_qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                      entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    ansatz.compose(init_state, front=True, inplace=True)
elif ansatz_type == "UCCSD":
    ansatz = UCCSD(problem.qubit_converter,problem.es_problem.num_particles,problem.es_problem.num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "PUCCD":
    ansatz = PUCCD(problem.qubit_converter,problem.es_problem.num_particles,problem.es_problem.num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "SUCCD":
    ansatz = SUCCD(problem.qubit_converter,problem.es_problem.num_particles,problem.es_problem.num_spin_orbitals,initial_state = init_state)

## Optimizer
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer_type = 'COBYLA'
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=500)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=500)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=500)

#how about freeze core, Z2symmetry reduction and removed orbitals?
if ansatz_type == "TwoLocal":
    param_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': problem.qubit_converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': rotation_blocks,
        'entanglement_blocks': entanglement_blocks,
        'entanglement': entanglement,
        'repetitions': repetitions,
        'skip_final_rotation_layer': skip_final_rotation_layer}
else:
    param_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': problem.qubit_converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': None,
        'entanglement_blocks': None,
        'entanglement': None,
        'repetitions': None,
        'skip_final_rotation_layer': None}

np.random.seed(5)  # fix seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)

iter_times = []
counts = []
values = []
params = []
deviation = []

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(problem.q_qubit_op)
exact_energy = problem.exact_energy
min_energy = np.real(result.eigenvalue)
optimal_state = result.eigenstate
optimal_point = result.optimal_point
percent_accuracy = 100-np.abs(100*(min_energy-exact_energy)/exact_energy)
nominal_accuracy = min_energy-exact_energy
min_energies = values

result_dict = {
    'molecule': mol_dict,
    'solution_type': method,
    'backend': str(backend),
    'setup' : param_dict,
    'exact energy': problem.exact_energy,
    'result energy (Ha)': min_energy,
    'error (mHa)': nominal_accuracy,
    'error (%)': percent_accuracy,
    'final parameters': optimal_point,
    '# of evaluations': len(min_energies),
    'runtime': result.optimizer_time,
    '# of qubits': int(problem.q_qubit_op.num_qubits),
    'min energies': min_energies}

minEnergyResults(result_dict, method)

'Evaluation: 42, Energy: -1.8624885042883244, Std: 0.0032008374035100494'

In [ ]:
### QCC method ###
#set number of Bloch angle and entangler amplitude foldings
method = 'QCC'
start_time = time.time()

angle_folds = 4
amplitude_folds = 1
m = count_qubits(problem.of_qubit_op)
#create dictionary of QubitOperator entanglers
entanglers = {'IYZI': QubitOperator('Y1 Z2'), 'IZYI': QubitOperator('Z1 Y2'),
              'IXYI': QubitOperator('X1 Y2'), 'IYXI': QubitOperator('Y1 X2')}

param_dict = {
    'mapping': problem.qubit_converter.mapper.__class__.__name__,
    'angle folds': angle_folds,
    'amplitudefolds': amplitude_folds,
    'entanglers': entanglers}

#run QCC method
min_energy, variables, min_energies, iter_times = QCC(problem.of_qubit_op, entanglers, angle_folds, amplitude_folds, sampler, num_cycles=10, num_samples=1000, strength=1e3, verbose=True)

exact_energy = problem.exact_energy
min_energy = np.real(min_energy)
optimal_point = variables
percent_accuracy = 100-np.abs(100*(min_energy-exact_energy)/exact_energy)
nominal_accuracy = min_energy-exact_energy

result_dict = {
    'molecule': mol_dict,
    'solution_type': method,
    'backend': str(sampler),
    'setup' : param_dict,
    'exact energy': exact_energy,
    'result energy (Ha)': min_energy,
    'error (mHa)': nominal_accuracy,
    'error (%)': percent_accuracy,
    'final parameters': optimal_point,
    '# of evaluations': len(min_energies),
    'runtime': sum(iter_times),
    '# of qubits': m,
    'min energies': min_energies}

minEnergyResults(result_dict, method)

-0.819476273820261 + 0.173455876425147*Z0*cos(the0) - 0.222425691975333*Z1*cos(the1) + 0.173455876425147*Z2*cos(the2) - 0.222425691975333*Z3*cos(the3) + (1 - cos(tau0))*(0.222425691975333*Z1*cos(the1) - 0.120779207929492*Z1*Z0*cos(the1)*cos(the0) - 0.166139664061611*Z2*Z1*cos(the1)*cos(the2) - 0.174855520523933*Z3*Z1*cos(the1)*cos(the3) - 0.0453604561321193*Q0*Q1*sin(the1)*sin(the2)*sin(the3)*sin(the0)*sin(phi1)*sin(phi0)*cos(phi3)*cos(phi2) - 0.0453604561321193*Q0*Q3*Q1*Q2*sin(the1)*sin(the2)*sin(the3)*sin(the0)*sin(phi3)*sin(phi1)*sin(phi0)*sin(phi2)) + (1 - cos(tau1))*(-0.173455876425147*Z2*cos(the2) - 0.168722640414494*Z2*Z0*cos(the0)*cos(the2) - 0.166139664061611*Z2*Z1*cos(the1)*cos(the2) - 0.120779207929492*Z3*Z2*cos(the3)*cos(the2) - 0.0453604561321193*Q3*Q2*sin(the1)*sin(the2)*sin(the3)*sin(the0)*sin(phi3)*cos(phi0)*sin(phi2)*cos(phi1) - 0.0453604561321193*Q0*Q3*Q1*Q2*sin(the1)*sin(the2)*sin(the3)*sin(the0)*sin(phi3)*sin(phi1)*sin(phi0)*sin(phi2)) + (1 - cos(tau2))*(0.222425691

TypeError: expected input vartype to be one of: Vartype.SPIN, 'SPIN', {-1, 1}, Vartype.BINARY, 'BINARY', or {0, 1}; received {(F2, F3): 2000.0, ('F3*F2', F3): 2000.0, ('F3*F2', F2): 2000.0, ('auxF3,F2', F3): 4000.0, ('auxF3,F2', F2): 4000.0, ('auxF3,F2', 'F3*F2'): 4000.0, (Z1, F3): -4.535179070861307e-18, (Z1, 'F3*F2'): 2000.0, ('Z1*F3*F2', 'F3*F2'): 2000.0, ('Z1*F3*F2', Z1): 2000.0, ('auxZ1,F3*F2', 'F3*F2'): 4000.0, ('auxZ1,F3*F2', Z1): 4000.0, ('auxZ1,F3*F2', 'Z1*F3*F2'): 4000.0, (Z2, F2): -2.7769962663451836e-34, (Z2, 'F3*F2'): 2000.0, (Z2, Z1): 4.0076655258636196e-24, ('Z2*F3*F2', 'F3*F2'): 2000.0, ('Z2*F3*F2', Z2): 2000.0, ('auxZ2,F3*F2', 'F3*F2'): 4000.0, ('auxZ2,F3*F2', Z2): 4000.0, ('auxZ2,F3*F2', 'Z2*F3*F2'): 4000.0, (F1, Z1): 2000.6938235057005, ('Z1*F1', Z1): 2000.0, ('Z1*F1', F1): 2000.0, ('auxZ1,F1', Z1): 4000.0, ('auxZ1,F1', F1): 4000.0, ('auxZ1,F1', 'Z1*F1'): 4000.0, (Z0, Z1): 2.958237407888099e-17, (Z0, 'Z1*F3*F2'): -6.103981476569903e-56, (Z0, Z2): 2.4921395617809655e-

In [ ]:
### XBK method ###
method = 'XBK'
start_time = time.time()
#set r value
r = 4

#construct qubit Hamiltonians and C terms for XBK method
qubit_Hs, qubit_Cs = [],[]
m = count_qubits(problem.of_qubit_op)
for p in range(int(math.ceil(r/2+1))):
    qubit_Hs += [XBK_transform(problem.of_qubit_op, r, p)]
    qubit_Cs += [construct_C(m, r, p)]

param_dict = {
    'mapping': problem.qubit_converter.mapper.__class__.__name__,
    'r': r}

#run XBK method
min_energy, ground_state, min_energies, iter_times = XBK(qubit_Hs, qubit_Cs, r, sampler, starting_lam=0, num_samples=1000, strength=1e3, verbose=True)

exact_energy = problem.exact_energy
min_energy = np.real(min_energy)
optimal_state = ground_state
percent_accuracy = 100-np.abs(100*(min_energy-exact_energy)/exact_energy)
nominal_accuracy = min_energy-exact_energy

result_dict = {
    'molecule': mol_dict,
    'solution_type': method,
    'backend': str(sampler),
    'setup' : param_dict,
    'exact energy': problem.exact_energy,
    'result energy (Ha)': min_energy,
    'error (mHa)': nominal_accuracy,
    'error (%)': percent_accuracy,
    'final parameters': 'N/A',
    '# of evaluations': len(min_energies),
    'runtime': sum(iter_times),
    '# of qubits': m,
    'min energies': min_energies}

minEnergyResults(result_dict, method)

KeyboardInterrupt: 

In [ ]:
result_dict

{'# of evaluations': 3,
 '# of qubits': 4,
 'backend': '<dwave.system.samplers.leap_hybrid_sampler.LeapHybridSampler object at 0x7f06cca9ac50>',
 'error (%)': 95.2986994609757,
 'error (mHa)': 0.08753928569029146,
 'exact energy': -1.8620227522926915,
 'final parameters': 'N/A',
 'min energies': [-1.7744834666024, -1.69371250410331, -1.63308714975331],
 'molecule': {'basis': 'sto6g', 'molecule name': 'H2'},
 'result energy (Ha)': -1.7744834666024,
 'runtime': 21.705110788345337,
 'setup': {'mapping': 'JordanWignerMapper', 'r': 4},
 'solution_type': 'XBK'}

In [ ]:
result_df = pd.DataFrame.from_dict(result_dict)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
print(str(sampler))
print(str(backend))

NameError: name 'backend' is not defined

In [ ]:
import json
with open('experiments/results.json', 'r') as file:
    results = json.load(file)
with open('experiments/results.json','w') as file:
        json.dump({"y":2}, file, sort_keys=True, indent=4)

In [ ]:
with open('results.json', 'r') as file:
    results = json.load(file)

results.update(result_dict)

with open('/content/drive/MyDrive/Woof/' + output_filename + ".json",'w') as file:
    json.dump(results, file, sort_keys=True, indent=4)

In [ ]:
mol_dict = {mol_name: mol_name,'basis': basis, 'q_fermionic_op' : str(q_fermionic_op.to_list())}

import json
with open(mol_name + '.json','w') as file:
      json.dump(mol_dict, file, sort_keys=True, indent=4)
        
qcc_op_list = list(str(qubit_op_to_expr(qubit_H)).split('+'))
print(*qcc_op_list, sep = "\n")   

properties = driver.run()
electronic_energy = properties.get_property(ElectronicEnergy)
particle_number = properties.get_property(ParticleNumber)
num_electron = sum(particle_number.num_particles)
num_MO = num_electron
num_SO = particle_number.num_spin_orbitals

#Properties
from qiskit_nature.properties import Property, GroupedProperty
from qiskit_nature.properties.second_quantization.electronic import (
    ElectronicEnergy,
    ElectronicDipoleMoment,
    ParticleNumber,
    AngularMomentum,
    Magnetization,
)
from qiskit_nature.properties.second_quantization.electronic.integrals import (
    ElectronicIntegrals,
    OneBodyElectronicIntegrals,
    TwoBodyElectronicIntegrals,
    IntegralProperty,
)
from qiskit_nature.properties.second_quantization.electronic.bases import ElectronicBasis